In [ ]:
%matplotlib inline
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
from torch.autograd import Variable
import nibabel as nib
import pandas as pd
from torch.utils import data
import random
from sklearn.model_selection import KFold
import argparse
import os
import time
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit
from scipy.spatial import distance_matrix
import pathlib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,balanced_accuracy_score,precision_score,f1_score,silhouette_score,recall_score,classification_report,confusion_matrix,roc_curve
from scipy.stats import ttest_ind
from mpl_toolkits.mplot3d import Axes3D   
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.decomposition import PCA
import seaborn as sns
from IPython.display import display, HTML
from scipy.stats import spearmanr,pearsonr, ttest_ind
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import pathlib
import sys
from IPython.display import display, HTML
from MyResNet import *
from scipy.ndimage import gaussian_filter
from nilearn import plotting

display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '#notebook-container {padding: 0px;}'
    '</style>'
))

In [ ]:
torch.cuda.empty_cache() 


class DatasetNii(Dataset):  
    def __init__(self, csv, data_pth):
        ### data is DataFrame with first column label and the rest SVD data
        self.data = pd.read_csv(csv)
        self.svd_arr = self.data['mri_ID']
        self.data_pth = data_pth

        self.data_len = len(self.data)

    def __getitem__(self, index):
        # read single nii
        single_pth = self.data_pth + self.svd_arr[index]
        single_data = nib.load(single_pth).get_fdata().astype(np.float32)
        single_data = np.expand_dims(single_data, axis = 0)
        # to tensor
        svd_as_tensor = torch.from_numpy(single_data)

        return svd_as_tensor, self.svd_arr[index]

    def __len__(self):
        return self.data_len
    
affine = np.eye(4)

In [ ]:
device = torch.device('cuda')
model_pth = '/media/dw/Data/BINC_T1/BINC/dropbox/DW/results_GenieCode/task2_T1_ADVDLBD_Mlabel_10fold_0307v2_3DSminmax_OTF/resnet18_64_linear_mlr0.0001_ep300_batch8_ASL_gn4_gp1_c0.1_sigma0.1_wd0.001/'
data_pth = '/media/dw/Data/BINC_T1/BINC/dropbox/nacc_harmonized_may023/beforeHarm_3DSminmax/'
data_pthUKBB = '/media/dw/Data/BINC_T1/BINC/dropbox/nacc_harmonized_may023/UKBB_3DSminmax/'
data_pathADNIlrp = '/media/dw/Data/BINC_T1/BINC/dropbox/nacc_harmonized_may023/ADNI_LRP_3DSminmax/'

# save_pth = "/home/dw/Desktop/DemCLF/model/results/task2_T1_ADVDLBD_Mlabel_10fold_NoHarm_3DSminmax_OTF/resnet18_64_linear_mlr0.0001_ep300_batch8_ASL_gn2_gp1_c0.2_sigma0.1/"

# MCI

In [ ]:
## average logit

def sig(x):
    return 1/(1+np.exp(-x))


## ADNI AD, CN from TW                                             NoD = NoDuplicates
test_ADNI = DatasetNii(csv = '/home/dw/Desktop/DemCLF/data/ADNI_TW_NoD.csv',  data_pth = '/media/dw/Data/BINC_T1/BINC/dropbox/nacc_harmonized_may023/ADNI_TW_3Dminmax/')
data_loader_ADNI = DataLoader(test_ADNI, batch_size = 16, num_workers = 0, shuffle = False)

########
all_ids = []
all_probAD = np.zeros(len(test_ADNI) ) 
all_probVD = np.zeros(len(test_ADNI) )
all_probLBD = np.zeros(len(test_ADNI) )

for i in range(0, 10): 
    f_probAD = []
    f_probVD = []
    f_probLBD = []
    print(i)
    model.load_state_dict(torch.load(model_pth + "/fold" + str(i) + ".pt"))
    model = model.to(device)

    all_prediction = []
    all_target = []
    
    model.eval()
    with torch.no_grad():

                
        for j, (data, id_) in enumerate(data_loader_ADNI):  
            model.eval()
            d = data.to(device)
            output = model(d).cpu().detach().numpy()
            f_probAD.extend(output[:,0])
            f_probVD.extend(output[:,1])
            f_probLBD.extend(output[:,2])
            if i == 0:
                all_ids.extend(id_)
                

    
    all_probAD += np.array(f_probAD)
    all_probVD += np.array(f_probVD)
    all_probLBD += np.array(f_probLBD)

    
all_probAD = all_probAD/10
all_probVD = all_probVD/10
all_probLBD = all_probLBD/10

all_probAD = [sig(i) for i in all_probAD]    
all_probVD = [sig(i) for i in all_probVD]  
all_probLBD = [sig(i) for i in all_probLBD]  

In [ ]:
df = pd.DataFrame(all_ids, columns = ['mri_ID'])
df.insert(loc = 1, column = 'probLBD', value = all_probLBD)
df.insert(loc = 1, column = 'probVD', value = all_probVD)
df.insert(loc = 1, column = 'probAD', value = all_probAD)
df.to_csv(model_pth + "/prob_ADNI.csv")
df

In [ ]:
df = pd.read_csv(model_pth + "/prob_ADNI.csv")
df.mri_ID = df.mri_ID.str[0:19]

a = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID",'DX', 'PTEDUCAT','MMSE']]
df = pd.merge(df,a, on='mri_ID', how='left')

df.to_csv(model_pth + "/prob_ADNI.csv")
df

In [ ]:
p = "/media/dw/Data/BINC_T1/BINC/dropbox/DW/results_GenieCode/task2_T1_ADVDLBD_Mlabel_10fold_0307v2_3DSminmax_OTF/resnet18_64_linear_mlr0.0001_ep300_batch8_ASL_gn4_gp1_c0.1_sigma0.1_wd0.001/"
df = pd.read_csv(p + "/prob_ADNI.csv")
b = pd.read_csv("/home/dw/Desktop/ADNI/ADNI_Scandates_and_Closest_Matching_Examdates_DW.csv").rename(columns = {'ID_SCANDATE' : 'mri_ID'})[["mri_ID", 'ABETA','AMYLOID_STATUS','AGE','PTGENDER']]
b = b.replace('<200', 200)
b = b.replace('>1700', 1701)
b['ABETA'] = b['ABETA'].astype(float)
df = pd.merge(df,b, on = 'mri_ID', how = 'left')
df = df.fillna(-4)
df = df.rename(columns = {'ABETA' : 'CSF_ABETA'})
df = df.rename(columns = {'AMYLOID_STATUS' : 'PET_ABETA'})


# df = df.drop(columns = ['MMSE'])
df_AD = df[df.DX == 'Dementia']
df_AD = df_AD[(df_AD['PET_ABETA']==1) |(df_AD['CSF_ABETA'] <= 978)]

df_mci = df[df.DX == 'MCI']
df_mci_p = df_mci[(df_mci['PET_ABETA']==1) | (df_mci['CSF_ABETA'] <= 978)]
# df_mci_p = df_mci[(df_mci['PET_ABETA']==1) ]
# df_mci_p = df_mci[(df_mci['CSF_ABETA'] <= 978)]

df_mci_n = df_mci[(df_mci['PET_ABETA']==0) & (df_mci['CSF_ABETA'] > 978)]
df_ADp = pd.concat([df_AD, df_mci_p], axis = 0)

df_CN = df[(df.DX == 'CN')]
df_CN = df_CN[(df_CN['PET_ABETA']==0)& (df_CN['CSF_ABETA'] > 978)]

print(len(df_AD),len(df_mci_p),len(df_mci_n),len(df_CN))

df_adni = pd.concat([df_ADp, df_mci_n, df_CN], axis = 0)

df_ukbb = pd.read_csv(p + "/prob_UKBB.csv")
df_CN


In [ ]:
print(len(df_adni),len(df_ADp),len(df_mci_n),len(df_CN))

print(np.mean(df_adni.AGE),np.std(df_adni.AGE))
print(np.mean(df_ADp.AGE),np.std(df_ADp.AGE))
print(np.mean(df_mci_n.AGE),np.std(df_mci_n.AGE))
print(np.mean(df_CN.AGE),np.std(df_CN.AGE))
print()
print(df_adni.PTGENDER.tolist().count('Female'),df_adni.PTGENDER.tolist().count('Male'))
print(df_ADp.PTGENDER.tolist().count('Female'),df_ADp.PTGENDER.tolist().count('Male'))
print(df_mci_n.PTGENDER.tolist().count('Female'),df_mci_n.PTGENDER.tolist().count('Male'))
print(df_CN.PTGENDER.tolist().count('Female'),df_CN.PTGENDER.tolist().count('Male'))
print()
print(np.mean(df_adni.MMSE),np.std(df_adni.MMSE))
print(np.mean(df_ADp.MMSE),np.std(df_ADp.MMSE))
print(np.mean(df_mci_n.MMSE),np.std(df_mci_n.MMSE))
print(np.mean(df_CN.MMSE),np.std(df_CN.MMSE))
print()
print(np.mean(df_adni.PTEDUCAT),np.std(df_adni.PTEDUCAT))
print(np.mean(df_ADp.PTEDUCAT),np.std(df_ADp.PTEDUCAT))
print(np.mean(df_mci_n.PTEDUCAT),np.std(df_mci_n.PTEDUCAT))
print(np.mean(df_CN.PTEDUCAT),np.std(df_CN.PTEDUCAT))

In [ ]:

l = len(df_ADp)
print('AD and MCI+', "total=",l)
print("predict CN = ",round(len(df_ADp[(df_ADp.probAD <=0.5)& (df_ADp.probVD <=0.5)&(df_ADp.probLBD <=0.5)])/l*100, 2),"%")
print("predict AD = ",round(len(df_ADp[(df_ADp.probAD >0.5)])/l*100,2),"%")
print("predict VD = ",round(len(df_ADp[(df_ADp.probVD >0.5)])/l*100,2),"%")
print("predict LBD = ",round(len(df_ADp[(df_ADp.probLBD >0.5)])/l*100,2),"%")


l = len(df_mci_n)
print('MCI-', "total=",l)
print("predict CN = ",round(len(df_mci_n[(df_mci_n.probAD <=0.5)& (df_mci_n.probVD <=0.5)&(df_mci_n.probLBD <=0.5)])/l*100, 2),"%")
print("predict AD = ",round(len(df_mci_n[(df_mci_n.probAD >0.5)])/l*100,2),"%")
print("predict VD = ",round(len(df_mci_n[(df_mci_n.probVD >0.5)])/l*100,2),"%")
print("predict LBD = ",round(len(df_mci_n[(df_mci_n.probLBD >0.5)])/l*100,2),"%")


l = len(df_CN)
print('CN', "total=",l)
print("predict CN = ",round(len(df_CN[(df_CN.probAD <=0.5)& (df_CN.probVD <=0.5)&(df_CN.probLBD <=0.5)])/l*100, 2),"%")
print("predict AD = ",round(len(df_CN[(df_CN.probAD >0.5)])/l*100,2),"%")
print("predict VD = ",round(len(df_CN[(df_CN.probVD >0.5)])/l*100,2),"%")
print("predict LBD = ",round(len(df_CN[(df_CN.probLBD >0.5)])/l*100,2),"%")

 
print(len(df_ukbb[(df_ukbb.probAD <=0.5)& (df_ukbb.probVD <=0.5)&(df_ukbb.probLBD <=0.5)])/3732)
print(len(df_ukbb[(df_ukbb.probAD >0.5)])/3732)
print(len(df_ukbb[(df_ukbb.probVD >0.5)])/3732)
print(len(df_ukbb[(df_ukbb.probLBD >0.5)])/3732)

In [ ]:
df = df_mci
fig,axs = plt.subplots(ncols = 3,nrows=4,figsize = (15,20))
axs = axs.flatten()

s = 100 
a = 0.4

f1,f2 ='probAD','probVD'
axs[0].scatter(df_ADp[[f1]], df_ADp[[f2]], marker='*', alpha = a, s =s, color = 'darkgreen')
axs[0].set_xlim([-0.05,1.05])
axs[0].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probLBD'
axs[1].scatter(df_ADp[[f1]], df_ADp[[f2]], marker='*', alpha = a, s =s, color = 'darkgreen')
axs[1].set_xlim([-0.05,1.05])
axs[1].set_ylim([-0.05,1.05])

f1,f2 ='probVD','probLBD'
axs[2].scatter(df_ADp[[f1]], df_ADp[[f2]], marker='*', alpha = a, s =s, color = 'darkgreen')
axs[2].set_xlim([-0.05,1.05])
axs[2].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probVD'
axs[3].scatter(df_mci_n[[f1]], df_mci_n[[f2]], marker='d', alpha = a, s =s, color = 'mediumseagreen')
axs[3].set_xlim([-0.05,1.05])
axs[3].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probLBD'
axs[4].scatter(df_mci_n[[f1]], df_mci_n[[f2]], marker='d', alpha = a, s =s, color = 'mediumseagreen')
axs[4].set_xlim([-0.05,1.05])
axs[4].set_ylim([-0.05,1.05])

f1,f2 ='probVD','probLBD'
axs[5].scatter(df_mci_n[[f1]], df_mci_n[[f2]], marker='d', alpha = a, s =s, color = 'mediumseagreen')
axs[5].set_xlim([-0.05,1.05])
axs[5].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probVD'
axs[6].scatter(df_CN[[f1]], df_CN[[f2]], marker='s', alpha = a, s = 50, color = 'gray')
axs[6].set_xlim([-0.05,1.05])
axs[6].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probLBD'
axs[7].scatter(df_CN[[f1]], df_CN[[f2]], marker='s', alpha = a, s = 50, color = 'gray')
axs[7].set_xlim([-0.05,1.05])
axs[7].set_ylim([-0.05,1.05])

f1,f2 ='probVD','probLBD'
axs[8].scatter(df_CN[[f1]], df_CN[[f2]], marker='s', alpha = a, s = 50, color = 'gray')
axs[8].set_xlim([-0.05,1.05])
axs[8].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probVD'
axs[9].scatter(df_ukbb[[f1]], df_ukbb[[f2]], marker='o', alpha = a, s = 50, color = 'gray')
axs[9].set_xlim([-0.05,1.05])
axs[9].set_ylim([-0.05,1.05])

f1,f2 ='probAD','probLBD'
axs[10].scatter(df_ukbb[[f1]], df_ukbb[[f2]], marker='o', alpha = a, s = 50, color = 'gray')
axs[10].set_xlim([-0.05,1.05])
axs[10].set_ylim([-0.05,1.05])

f1,f2 ='probVD','probLBD'
axs[11].scatter(df_ukbb[[f1]], df_ukbb[[f2]], marker='o', alpha = a, s = 50, color = 'gray')
axs[11].set_xlim([-0.05,1.05])
axs[11].set_ylim([-0.05,1.05])

# axs[2].legend(prop={'size': 30})

# axs[2].scatter(-100,-100, label = "ADNI AD & MCI+", marker='*', alpha = a, s =s, color = 'darkgreen')
# axs[2].scatter(-100,-100, label = "ADNI MCI-", marker='d', alpha = a, s = s, color = 'mediumseagreen')
# axs[2].scatter(-100,-100, label = "ADNI CN", marker='s', alpha = a, s =50, color = 'gray')
# axs[2].scatter(-100,-100, label = "UKBB", marker='o', alpha = a, s =50, color = 'gray')
# axs[2].legend(prop={'size': 20},bbox_to_anchor=(1,1, 0.5, 0.5))

for i, j,k in zip(range(0,12), ['AD','AD','VD']*4, ['VD','LBD', 'LBD']*4):
    axs[i].axvline(x = 0.5, linestyle = '--', color='gray', alpha = 0.5)
    axs[i].axhline(y = 0.5, linestyle = '--', color='gray', alpha = 0.5)
    
    axs[i].set_xlabel(r'$\bf{'+j+'}$ Probability', fontsize = 20)
    axs[i].set_ylabel(r'$\bf{'+k+'}$ Probability', fontsize = 20) 


    
    axs[i].set_xticklabels([f'{x:.0%}' for x in plt.gca().get_xticks()], fontsize=12) 
    axs[i].set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()], fontsize=12) 
    
fig.tight_layout()